In [1]:
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import optimizers
import keras
import pandas
import utils

Using TensorFlow backend.


In [8]:
train_folder = 'data/train'
test_folder = 'data/test'
validation_folder = 'data/validation'
labels_csv = 'train_labels.csv'
label_names = ['0', '1']
IMG_WIDTH = 193
IMG_HEIGHT = 144
batch_size = 16

In [3]:
utils.create_labelled_folders(train_folder, labels_csv, label_names)        
utils.create_validation_subfolders(train_folder, validation_folder, label_names, 0.2)        

In [4]:
train_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    rescale=1./255.,
    horizontal_flip=True,
)

validation_datagen = ImageDataGenerator(
    rescale=1./255.
)

train_generator = train_datagen.flow_from_directory(
    train_folder,
    batch_size=batch_size,
    target_size = (IMG_WIDTH, IMG_HEIGHT)
)

validation_generator = validation_datagen.flow_from_directory(
    validation_folder,
    batch_size=batch_size,
    target_size = (IMG_WIDTH, IMG_HEIGHT)
)

Found 1837 images belonging to 2 classes.
Found 458 images belonging to 2 classes.


In [11]:
base_model = keras.applications.vgg19.VGG19(weights='imagenet', 
                                            include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

78823424/80134624 [============================>.] - ETA: 0s

In [12]:
x = base_model.output
x = (Flatten(input_shape=base_model.output_shape[1:], name="finetune"))(x)
x = (Dense(256, activation='relu'))(x)
x = (Dropout(0.5))(x)
x = (Dense(2, activation='sigmoid'))(x)

model = Model(inputs=base_model.inputs, outputs=x)

for layer in model.layers:
    if layer.name == "finetune":
        break
    
    layer.trainable = False

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [14]:
num_training_samples = train_generator.n
num_validation_samples = validation_generator.n

model.fit_generator(
    train_generator,
    steps_per_epoch=num_training_samples//batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=num_validation_samples//batch_size)

Epoch 1/50
114/114 [==============================] - 19s - loss: 0.4675 - acc: 0.7588 - val_loss: 0.3495 - val_acc: 0.8884
Epoch 2/50
114/114 [==============================] - 17s - loss: 0.4165 - acc: 0.8081 - val_loss: 0.3313 - val_acc: 0.8864
Epoch 3/50
114/114 [==============================] - 17s - loss: 0.4005 - acc: 0.8224 - val_loss: 0.3823 - val_acc: 0.8348
Epoch 4/50
114/114 [==============================] - 17s - loss: 0.4488 - acc: 0.7741 - val_loss: 0.3599 - val_acc: 0.8795
Epoch 5/50
114/114 [==============================] - 17s - loss: 0.4528 - acc: 0.7697 - val_loss: 0.3695 - val_acc: 0.8571
Epoch 6/50
114/114 [==============================] - 17s - loss: 0.4200 - acc: 0.8191 - val_loss: 0.3873 - val_acc: 0.8227
Epoch 7/50
114/114 [==============================] - 17s - loss: 0.4187 - acc: 0.8026 - val_loss: 0.3312 - val_acc: 0.8929
Epoch 8/50
114/114 [==============================] - 17s - loss: 0.3850 - acc: 0.8202 - val_loss: 0.3250 - val_acc: 0.8616
Epoch 9/

In [ ]:
test_datagen = ImageDataGenerator(
    rescale=1./255.
)

test_generator = test_datagen.flow_from_directory(
    test_folder,
    batch_size=batch_size,
    target_size = (IMG_WIDTH, IMG_HEIGHT)
)

In [ ]:
num_test_samples = test_generator.n
test_predictions = model.predict_generator(test_generator, num_test_samples//batch_size)

In [ ]:
test_predictions[:,:20]